In [1]:
import numpy as np
import pickle
import sys

In [2]:
BOARD_ROWS = 7
BOARD_COLS = 7

In [7]:
class State:
  def __init__(self, p1, p2):
    self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
    self.board[0, 0] = self.board[BOARD_ROWS-1, BOARD_COLS-1] = 1
    self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = -1
    self.p1 = p1
    self.p2 = p2
    self.isEnd = False
    self.boardHash = None
    # init p1 plays first
    self.playerSymbol = 1

  def cantmove(self):
    for i in range(BOARD_ROWS):
      for j in range(BOARD_COLS):
        if self.board[i, j] == 0:
          self.board[i, j] = -self.playerSymbol
    return None

  def winner(self):
    if sum(map(sum, map(abs, self.board))) == BOARD_ROWS*BOARD_COLS:
      self.isEnd = True
      return sum(map(sum, self.board))
    self.isEnd = False
    return None

  def availableActions(self):
    Actions = []
    for i in range(BOARD_ROWS):
        for j in range(BOARD_COLS):
            if self.board[i, j] == self.playerSymbol:
                for ii in range(-2,3):
                    for jj in range(-2,3):
                        if ii == 0 and jj == 0:
                          continue
                        if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                          continue
                        if self.board[i + ii, j + jj] == 0:
                          act = i
                          act = act*BOARD_COLS + j
                          act = act*BOARD_COLS + i + ii
                          act = act*BOARD_COLS + j + jj
                          Actions.append(act)
    return Actions

    def updateState(self, Action):
        position = []
        for i in range(4):
          position[3-i] = Action % BOARD_COLS
          Action /= BOARD_COLS
        ii = position[2] - position[0]
        jj = position[3] - position[1]
        if max(abs(ii), abs(jj)) == 2:
            self.board[position[0:2]] = 0
        
        dx1 = [-1, -1, -1, 0, 0, 1, 1, 1]
        dy1 = [-1, 0, 1, -1, 1, -1, 0, 1]
        i, j = position[2:4]
        self.board[i, j] = self.playerSymbol
        for ii, jj in zip(dx1, dy1):
            if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                continue
            if self.board[i + ii, j + jj] == -self.playerSymbol:
                self.board[i + ii, j + jj] = self.playerSymbol
            
        # switch to another player
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1

    def giveReward(self):
        result = self.winner()
        # backpropagate reward
        self.p1.feedReward(result)
        self.p2.feedReward(-result)

    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.board[0, 0] = self.board[BOARD_ROWS-1, BOARD_COLS-1] = 1
        self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = -1
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1

    

In [15]:
class Player:
  def __init__(self, name, exp_rate=0.1):
    self.name = name
    self.lr = 0.2
    self.exp_rate = exp_rate
    self.decay_gamma = 0.95

  def build_model(self):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(7, 7), activation='relu'))
    model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(5, 5), activation='relu'))
    model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(3, 3), activation='relu'))
    model.add(Dense(64 * BOARD_COLS * BOARD_COLS, activation='relu'))
    model.add(Dense(BOARD_COLS**4, activation='relu'))
    print(model.summary())
    model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
    return model

  def getAction(self, state):
    if np.random.rand() <= self.epsilon
       # 무작위 행동 반환
      return random.randrange(self.action_size)
    else:
       # 모델로부터 행동 산출
      state = np.float32(state)
      q_values = self.model.predict(state)
      return np.argmax(q_values[0])

  def train_model(self):


In [16]:
if __name__ ==" __main__":
  agent = Player()
  agent.build_model()